In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from utils import set_seed
from utils import sample
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
        
        print(chars)
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [5]:
block_size = 128 # spatial extent of the model for its context

In [6]:
# you can download this file at https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
text = open('kara_sozder_abay_joly.txt', 'r', encoding='utf-8').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

data has 3493140 characters, 145 unique.
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'C', 'H', 'I', 'M', 'T', 'V', 'X', '^', 'a', 'b', 'c', 'e', 'f', 'h', 'i', 'l', 'n', 'o', 'p', 's', 't', 'y', '~', '£', '¥', '«', '»', 'Ё', 'І', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ч', 'Ш', 'Щ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'і', 'Ғ', 'ғ', 'Қ', 'қ', 'Ң', 'ң', 'Ү', 'ү', 'Ұ', 'ұ', 'Һ', 'һ', 'Ә', 'ә', 'Ө', 'ө', '–', '—', '„', '…', '№', '™']


In [7]:
from model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=3, n_head=3, n_embd=48)
model = GPT(mconf)

05/12/2023 11:00:05 - INFO - model -   number of parameters: 2.524320e+05


In [8]:
from trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=5, batch_size=512, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=0)
trainer = Trainer(model, train_dataset, None, tconf)
#trainer.train()

# Save model

In [9]:
SAVE_PATH = 'gpt200k.pt'

In [10]:
torch.save(model.state_dict(), SAVE_PATH)

# Load model

# Error caused by tokens mismatching, text must be Abay zholy + kara_sozder

In [11]:
model = GPT(mconf)
model.load_state_dict(torch.load('gpt200k_abay.pt'))
model.eval().cuda()

05/12/2023 11:00:07 - INFO - model -   number of parameters: 2.524320e+05


GPT(
  (tok_emb): Embedding(145, 48)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=48, out_features=48, bias=True)
        (query): Linear(in_features=48, out_features=48, bias=True)
        (value): Linear(in_features=48, out_features=48, bias=True)
        (attn_drop): Dropout(p=0.1, inplace=False)
        (resid_drop): Dropout(p=0.1, inplace=False)
        (proj): Linear(in_features=48, out_features=48, bias=True)
        (time_shift): ZeroPad2d((0, 0, 1, 0))
      )
      (mlp): Sequential(
        (0): Linear(in_features=48, out_features=192, bias=True)
        (1): GELU(approximate=none)
        (2): Linear(in_features=192, out_features=48, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((48

In [122]:
context = "Абай өзінен үлкен кісінің қорыққанына қатты ырза еді. Жұмабайдың неге ашуланғанын ұғып тұр. Қоңыр жүзі қызарып, төмен қарап қысыла күле беріп, бөркін айналдыра бастады. Кәдімгі «жолбасар» ұрыларша шапан-бөркін айналдырып киіп, мұрны мен аузын қызыл орамалмен таңып алып, Жұмабайды қуғанда тағы сол ұрыларша, «даусымды танытпаймын» деп, мыңқылдап сөйлеп бұйрық берген. Байтас қорықса, қорықпаса да сыр алдырған жоқ Сондықтан, Жұмабайдың ашуын алыстан танып, мәз болып күліп келе"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 10, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

Абай өзінен үлкен кісінің қорыққанына қатты ырза еді. Жұмабайдың неге ашуланғанын ұғып тұр. Қоңыр жүзі қызарып, төмен қарап қысыла күле беріп, бөркін айналдыра бастады. Кәдімгі «жолбасар» ұрыларша шапан-бөркін айналдырып киіп, мұрны мен аузын қызыл орамалмен таңып алып, Жұмабайды қуғанда тағы сол ұрыларша, «даусымды танытпаймын» деп, мыңқылдап сөйлеп бұйрық берген. Байтас қорықса, қорықпаса да сыр алдырған жоқ Сондықтан, Жұмабайдың ашуын алыстан танып, мәз болып күліп келедi. Былай 


In [12]:
# from utils import sample

context = "Мен "
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 400, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

Мен қазір астарында жазылып келе жатыр. Сонымен тағы тірлігінің қайтты десе, жөнін бір барлық болатын. Оның барын болғанын еститін деп болсын, сөзін жөндеп алып, бір екеуіне онша бұл жиын осылай жарқапты. Әйгерім екеуі қара тұсынан ойланғандай боп, қоныс жеңілген байын. Соны да алдында барын бастаған-ды.
Бұл жүз бала екі шайсыз таңғаны қозылысқан бақылды соның жеті ашылуы бойынан тағы бір күн құйдырып


In [100]:
context = "Сен "
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 30, temperature=1, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

Сен аулы күңкетпестен төр жүз алып
